In [1]:
import os
import sys
import bisect

In [15]:
class SpellChecker:
    def __init__(self, dict_path):
        self.dict_path  = dict_path
        self.word_dict = self.load_dict(self.dict_path)
    
    
    def store_words(self, word_list):
        word_dict = {}
        
        for word in word_list:
            key = word[0]
            if key not in word_dict.keys():
                word_dict[key] = []
            word_dict[key].append(word)
            
        for key in word_dict.keys():
            word_dict[key] = sorted(word_dict[key])
        return word_dict
    
    def load_dict(self, dict_path):
        word_list = []
        with open(dict_path,errors="ignore") as file:
            word_list = file.read().splitlines()
            
        return self.store_words(word_list)
    
    def add(self, word):
        key = word[0]
        self.word_dict[key].append(word)
        self.word_dict[key] = sorted(self.word_dict[key])
    
    def get_nearest(self, word, nearest = 4):
        nearest_words = []
        key = word[0]
        
        if word in self.word_dict[key]:
            return [word]
        
        augmented_sublist = self.word_dict[key].copy()
        target_idx = bisect.bisect_left(augmented_sublist, word)
        
        if target_idx < 2:
            nearest_words = augmented_sublist[:target_idx+2]
            
            prev_key = chr(ord(key) - 1)
            while prev_key in self.word_dict.keys():
                for word in self.word_dict[next_key]:
                    nearest_words.append(word)
                    if len(nearest_words) == 4:
                        return nearest_words
                prev_key = chr(ord(prev_key) - 1)
            return nearest_words
        
        elif target_idx > len(augmented_sublist) - 1:
            nearest_words = augmented_sublist[target_idx:]
            
            next_key = chr(ord(key)+1)
            while next_key in self.word_dict.keys():
                for word in self.word_dict[next_key]:
                    nearest_words.append(word)
                    if len(nearest_words) == 4:
                        return nearest_words
                next_key = chr(ord(next_key)+1)
            return nearest_words
            
        else:
            nearest_words = augmented_sublist[target_idx-2:target_idx+2]
            return nearest_words

In [16]:
sc = SpellChecker('./dictionary.txt')

In [17]:
sc.word_dict['ƒ']

['ƒclat', 'ƒlan', 'ƒmigrƒs', 'ƒpoque']

In [18]:
sc.add('ƒaaaa')

In [19]:
sc.word_dict['ƒ']

['ƒaaaa', 'ƒclat', 'ƒlan', 'ƒmigrƒs', 'ƒpoque']

In [20]:
sc.get_nearest('ƒaaa')

1


['ƒaaaa', 'ƒclat']

# Discussion

The data structure used to store the dictionary is the same as real language dictionaries. In other words, A hash-map where the key is the first letter of the word and the value is array containing all words starts with this letter.

    - dictionary = {
                    'a' : ['aa',..., 'aaes'],
                    'b' : ['bb',...,'bsfda'],
                     .
                     .
                     .
                     'z':['zads',...,'zasd'],
                    }
                    
reasons to use this data-structure:
        - Search operation in hash-map is O(1), so using hash-map will be suitable for search operations.
        - words can't be keys themselves as hash-map are (keys, values)
        - Having the values as arrays help us in traversing the dictionary in lexographically order. This can be done by sorting each value array. While the sorting of the keys can be infered --> the values of 'a' becomes earlier than values of 'c'.
        

complexity of operations:
    - add: O(1). Since the list is dynamic in python, the cost of n .append() operations is n. In other words, the amortized cost is O(1).
    
    - get_nearest: if word is not in the dictionary,
         - copy the target list --> O(n) ; n is the number of words starts with same start character of passed word
         - bisect_left uses binary search --> O(log n)
         - best case: the 4 nearest words in the target array --> slicing the array O(n+4)
           other cases: one or two of the nearest words are in previous or next characters --> a while loop that in worst case will be of constant iterations and constant .append() operations --> ~ O(n+4)
        
        -over all complexity O(n)

# Other approaches

Other approaches will be manily revolving around which data-structure used to store the dictionary. Other methods will depend on that choice. So. let's discuss other data-structures:

1- List
     - if all the words are stored in one list, the search process will be O(n) where n is the total number of words in the dictionary. Obviously, a bas idea.
     
2- Hashmap(dictionary)
    - every words is a key, so search is of O(1). But no sorting is guranteed. not feasible for the task
    
3- Nested dictionaries
    - Level 1: where key is fist letter and value is list of all words begin with this letter. (used)
    - Level 2: where for each key there is another dictionaries. This will make search for a word faster, but searching lexographically is more exhausting, and more memory is used.